In [ ]:
#This Stack Requires:
# 1. Docker
# 2. Jupyter
# 3. Selenium
# 4. RSelenium

#Practically, I wanted to create a predictor that links the year, the content, and the 3rd party views on youtube.
#I wanted to practice selinum scraping in R, and this seemed a perfect example as this already has links


In [ ]:
#Start your Selinum Instance Now
#docker run -d -p 4444:4444 --shm-size="2g" selenium/standalone-chrome:4.1.2-20220131

In [ ]:
use_docker <- TRUE # Uses docker to pull and augment data, requires the Selenium Instance
use_dev <- TRUE #Regenerates all values, rather than pulling from final values
Fresh_Data <- TRUE
Fresh_Computes <- TRUE
Dev_Checks <- TRUE

In [5]:
install.packages("tidyverse")
library(tidyverse)

also installing the dependencies 'xfun', 'glue', 'lifecycle', 'tidyselect', 'cli', 'gargle', 'vroom', 'knitr', 'broom', 'dplyr', 'dtplyr', 'googledrive', 'googlesheets4', 'ggplot2', 'haven', 'hms', 'jsonlite', 'lubridate', 'pillar', 'purrr', 'readr', 'reprex', 'rlang', 'rvest', 'tibble', 'tidyr', 'xml2'




  There are binary versions available but the source versions are later:
              binary source needs_compilation
xfun            0.22   0.29              TRUE
glue           1.4.2  1.6.1              TRUE
lifecycle      1.0.0  1.0.1             FALSE
cli            2.5.0  3.1.1              TRUE
gargle         1.1.0  1.2.0             FALSE
vroom          1.4.0  1.5.7              TRUE
knitr           1.33   1.37             FALSE
broom          0.7.6 0.7.12             FALSE
dplyr          1.0.6  1.0.7              TRUE
dtplyr         1.1.0  1.2.1             FALSE
googledrive    1.0.1  2.0.0             FALSE
googlesheets4  0.3.0  1.0.0             FALSE
ggplot2        3.3.3  3.3.5             FALSE
haven          2.4.1  2.4.3              TRUE
hms            1.0.0  1.1.1             FALSE
jsonlite       1.7.2  1.7.3              TRUE
lubridate     1.7.10  1.8.0              TRUE
pillar         1.6.0  1.7.0             FALSE
readr          1.4.0  2.1.2              TRUE
reprex

Warning message:
"cannot remove prior installation of package 'glue'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Aman\anaconda3\envs\Test\Lib\R\library\00LOCK\glue\libs\x64\glue.dll to C:\Users\Aman\anaconda3\envs\Test\Lib\R\library\glue\libs\x64\glue.dll: Permission denied"Warning message:
"restored 'glue'"

package 'tidyselect' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'tidyselect'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Aman\anaconda3\envs\Test\Lib\R\library\00LOCK\tidyselect\libs\x64\tidyselect.dll to C:\Users\Aman\anaconda3\envs\Test\Lib\R\library\tidyselect\libs\x64\tidyselect.dll: Permission denied"Warning message:
"restored 'tidyselect'"

...

package 'xml2' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'xml2'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Aman\anaconda3\envs\Test\Lib\R\library\00LOCK\xml2\libs\x64\xml2.dll to C:\Users\Aman\anaconda3\envs\Test\Lib\R\library\xml2\libs\x64\xml2.dll: Permission denied"Warning message:
"restored 'xml2'"

package 'tidyverse' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Aman\AppData\Local\Temp\RtmpIxsp1X\downloaded_packages


installing the source packages 'lifecycle', 'gargle', 'knitr', 'broom', 'dtplyr', 'googledrive', 'googlesheets4', 'ggplot2', 'hms', 'pillar', 'reprex', 'rvest'

Warning message in install.packages("tidyverse"):
"installation of package 'lifecycle' had non-zero exit status"Warning message in install.packages("tidyverse"):
"installation of package 'gargle' had non-zero exit status"Warning message in install.packages("tidyverse"):
"installation of package 'knitr' had non-zero exit status"Warning message in install.packages("tidyverse"):
"installation of package 'ggplot2' had non-zero exit status"Warning message in install.packages("tidyverse"):
"installation of package 'broom' had non-zero exit status"Warning message in install.packages("tidyverse"):
"installation of package 'dtplyr' had non-zero exit status"Warning message in install.packages("tidyverse"):
"installation of package 'hms' had non-zero exit status"Warning message in install.packages("tidyverse"):
"installation of package 'p

ERROR: Error: package or namespace load failed for 'tidyverse' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'broom' 0.5.2 is being loaded, but >= 0.7.6 is required


In [ ]:
url.data <- "https://raw.githubusercontent.com/fivethirtyeight/superbowl-ads/main/superbowl-ads.csv"

raw <- read.csv(url(url.data), header = TRUE,) 

In [ ]:
head(raw)

In [ ]:
#This is the raw data from fivethirtyeight
#Now we're going to augment it

In [ ]:
if (Fresh_Computes && Fresh_Data){
    remDr <- RSelenium::remoteDriver(remoteServerAddr = "localhost",
                                 port = 4444,
                                 browserName = "chrome")
    remDr$open()
}

In [ ]:
print(raw[3,]['brand'])


In [ ]:
if (Fresh_Computes && Fresh_Data && Dev_Checks){
    remDr$navigate("https://www.youtube.com/watch?v=zeBZvwYQ-hA") #Entering our URL gets the browser to navigate to the page
    remDr$screenshot(display = TRUE)
}

In [ ]:
if (Fresh_Computes && Fresh_Data){
    webElem <- remDr$findElement(using = "css", "[class='view-count style-scope ytd-video-view-count-renderer']")
    view_number_text = webElem$getElementText()
    view_number_text = gsub("views", "", view_number_text)
    view_number_text = gsub(",", "", view_number_text)
    print(view_number_text)
}

In [ ]:
wetest <- function(sleepmin,sleepmax){
  remDr <- get("remDr",envir=globalenv())
  webElemtest <-NULL
  while(is.null(webElem)){
    webElem <- tryCatch({remDr$findElement(using = "css", "[class='view-count style-scope ytd-video-view-count-renderer']")},
                            error = function(e){NULL})
    #loop until element with name <value> is found in <webpage url>
  }
  rsleep <- sample(seq(sleepmin, sleepmax, by = 0.001), 1)
  Sys.sleep(rsleep)
}


In [ ]:
if (Dev_Checks){
    for (url in raw$youtube_url) {
        remDr$navigate(url)      
        print(remDr$getCurrentUrl())
        wetest(sleepmin=.5,sleepmax=1)
        #webElem <- remDr$findElement(using = "css", "[class='view-count style-scope ytd-video-view-count-renderer']")
        view_number_text = webElem$getElementText()
        view_number_text = gsub("views", "", view_number_text)
        view_number_text = gsub(",", "", view_number_text)
        print(view_number_text)
    }
}

In [ ]:
c_dat =  select(raw, -youtube_url, -superbowl_ads_dot_com_url)

In [22]:
head(c_dat)

year,brand,funny,show_product_quickly,patriotic,celebrity,danger,animals,use_sex
2018,Toyota,False,False,False,False,False,False,False
2020,Bud Light,True,True,False,True,True,False,False
2006,Bud Light,True,False,False,False,True,True,False
2018,Hynudai,False,True,False,False,False,False,False
2003,Bud Light,True,True,False,False,True,True,True
2020,Toyota,True,True,False,True,True,True,False


In [24]:
unique(c_dat['brand'])

,brand
1,Toyota
2,Bud Light
4,Hynudai
7,Coca-Cola
8,Kia
10,Budweiser
15,NFL
18,Pepsi
20,Doritos
34,E-Trade


In [26]:
install.packages("RSelenium")

also installing the dependencies 'bitops', 'semver', 'XML', 'caTools', 'wdman', 'binman'



package 'bitops' successfully unpacked and MD5 sums checked
package 'semver' successfully unpacked and MD5 sums checked
package 'XML' successfully unpacked and MD5 sums checked
package 'caTools' successfully unpacked and MD5 sums checked
package 'wdman' successfully unpacked and MD5 sums checked
package 'binman' successfully unpacked and MD5 sums checked
package 'RSelenium' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Aman\AppData\Local\Temp\RtmpOi363W\downloaded_packages


In [29]:
library('RSelenium')
